# Finetuning RoBERTa for NER: Evaluate Model
 

***

## Imports

In [29]:
from transformers import (BertTokenizerFast,
                          RobertaTokenizerFast,
                          AutoTokenizer,
                          BertForTokenClassification,
                          RobertaForTokenClassification,
                          DataCollatorForTokenClassification, 
                          AutoModelForTokenClassification, 
                          TrainingArguments, Trainer)
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict
from pprint import pprint
import numpy as np
import pickle
import torch
import os

## Load Dataset

In [30]:
data_path = "./data/dataset_processed.pkl"
with open(data_path, 'rb') as pickle_file:
    dataset = pickle.load(file=pickle_file)

## Load Model and Tokenizer

Information about model variants can be found here: https://huggingface.co/docs/transformers/model_doc/roberta

Load Model which was finetuned:

In [31]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [32]:
label_list = dataset["train"].features[f"ner_tags"].feature.names

In [33]:
# model_name = "xlm-roberta-large" #"bert-base-multilingual-cased" #xlm-roberta-large
tokenizer = AutoTokenizer.from_pretrained("./results/checkpoint-final/", add_prefix_space=True) #AutoTokenizer(use_fast = True)
model = AutoModelForTokenClassification.from_pretrained("./results/checkpoint-final/")

loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file ./results/checkpoint-final/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./results/checkpoint-final/",
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 5,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_atte

**Define Metrics:**

See https://huggingface.co/course/chapter7/2#metrics

In [34]:
metric = load_metric("seqeval")

In [35]:
print(dataset["train"][150])

{'tokens': ["''", 'Psolos', 'fuligo', 'fuligo', "''", '—', "''", "'Coon", "''", "'"], 'ner_tags': [0, 5, 6, 6, 0, 0, 0, 0, 0, 0], 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], 'spans': ['LOC: Psolos fuligo fuligo'], 'input_ids': [5106, 436, 112032, 7, 45201, 20828, 45201, 20828, 5106, 292, 5106, 242, 10625, 191, 5106, 242], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 5, 5, 5, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0]}


In [36]:
example = dataset["train"][150]
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

**Calculate Accuracy:**

In [37]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
predictions, labels, _ = trainer.predict(dataset["test"])
predictions = np.argmax(predictions, axis=-1)

The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tokens, spans, langs, ner_tags. If tokens, spans, langs, ner_tags are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 16
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/pop529700/.pyenv/versions/3.10.8/envs/transformers/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [39]:
label_names = dataset["train"].features[f"ner_tags"].feature.names

In [40]:
true_labels = [
    [label_names[l] for l in label  if l != -100] 
    for label in labels
]

true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label)  if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
pprint(results)

{'LOC': {'f1': 0.5492468134414833,
         'number': 880,
         'precision': 0.5602836879432624,
         'recall': 0.5386363636363637},
 'ORG': {'f1': 0.40428211586901763,
         'number': 758,
         'precision': 0.38674698795180723,
         'recall': 0.4234828496042216},
 'PER': {'f1': 0.6785508913168488,
         'number': 782,
         'precision': 0.6165099268547545,
         'recall': 0.7544757033248082},
 'overall_accuracy': 0.8205149229255494,
 'overall_f1': 0.5481891945378983,
 'overall_precision': 0.5260159513862515,
 'overall_recall': 0.5723140495867769}
